In [ ]:
'This cell imports necessary libraries'
import numpy as np
import pandas as pd
from collections import Counter
filepath = '/home/syshain/Arshad/Lear_India_Remote_Internship/Task2_OEE/HSL1/'

In [ ]:
'This cell imports the dataset'
dataset = pd.read_csv(filepath + 'HSL1_Report_modified_for_OEE.csv')
date = dataset['Date']
times = dataset['Time']
result = dataset[' Result']

In [ ]:
'This cell calculates the cycle time'
del_time = pd.to_timedelta(times)
time_differences = np.asarray([del_time[i+1]-del_time[i] for i in range(len(del_time)-1)])
max_rep = Counter(time_differences).most_common(4)
'This line finds the most common time difference which can be taken as the ideal time required to produce one set of LH and RH recliners'
cycle_time = (max_rep[1][0]).total_seconds()/4

In [ ]:
'This cell calculates all intermediate quantities'
unplanned_dt = time_differences.sum(initial = pd.to_timedelta('00:00:00'), where = (time_differences > pd.to_timedelta('00:01:00')))
no_ok = Counter(result)['OK']
no_ng = Counter(result)['NG']
total_possibility = 24*3600/cycle_time

In [ ]:
'This cell calculates all final quantities'
availability = 1 - unplanned_dt/pd.to_timedelta('24:00:00')
quality = no_ok/(no_ok + no_ng)
OEE = no_ok/total_possibility
performance = OEE/(availability*quality)

In [ ]:
'This cell prints all the results'
print('The required results from the calculations are as follow:' + '\n' + 'OEE: ' + str(OEE) + '\n'+ 'Availability: ' + str(availability) + '\n' + 'Performance: ' + str(performance) + '\n' + 'Quality: ' + str(quality))

In [ ]:
'This cell prepares the dataset for calculation of hourly quantities'
timear = pd.to_datetime(dataset['Date'] + ' ' + dataset['Time'])
hourly_distribution = []
result_hrly = []
hrly_diff = []
blank = 0
onehr = pd.to_timedelta('1:00:00')
starttime = pd.to_datetime('15-10-2019'+' ' + '7:00:00')

In [ ]:
'This cell contains a loop to continue quantity preparation and converts them to appropriate types'
for i in range(17):
    hourly_distribution.append(timear[np.where(timear > starttime+i*(onehr))[0][0]:(np.where(timear > starttime+(i+1)*(onehr))[0][0])-1])
    result_hrly.append(result[np.where(timear > starttime+i*(onehr))[0][0]:(np.where(timear > starttime+(i+1)*(onehr))[0][0])-1])
    hrly_diff.append([hourly_distribution[i][j+1] - hourly_distribution[i][j] for j in range(blank, blank + len(hourly_distribution[i])-1)])
    blank = (np.where(timear > starttime+(i+1)*(onehr))[0][0])
    print(blank)

In [ ]:
'This cell calculates the most important quantities relating to the final calculations'
hrly_diff = np.array(hrly_diff)
hrly_unplanned = []
for i in range(17):
    hrly_unplanned.append(np.array(hrly_diff[i]).sum(initial = pd.to_timedelta('00:00:00'), where  = (np.array(hrly_diff[i]) > pd.to_timedelta('00:01:00'))))
ok_hrly = np.array([Counter(result_hrly[i])['OK'] for i in range(len(result_hrly))])
ng_hrly = np.array([Counter(result_hrly[i])['NG'] for i in range(len(result_hrly))])
hrly_possibility = total_possibility/24;

In [ ]:
'This cell calculates all the final hourly quantities'
availability_hrly = 1 - np.divide(hrly_unplanned,pd.to_timedelta('1:00:00'))
quality_hrly = np.divide(ok_hrly,(ok_hrly + ng_hrly))
OEE_hrly = ok_hrly / hrly_possibility
performance_hrly = OEE_hrly/(availability_hrly*quality_hrly)